
# Modelo Predictivo de Ingresos Mensuales con Datos Reales - Proyecto Gym Master

## Objetivo
Construir un modelo de machine learning que prediga los ingresos mensuales futuros del gimnasio usando los datos reales de segmentación y pagos.


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (12,6)


In [ ]:

# Cargar datos de segmentación real
segmentos_df = pd.read_csv('../../Data_Lake_CSV/segmentacion_socios.csv')

# Simular histórico de ingresos por mes basado en datos reales
np.random.seed(42)
meses = pd.date_range(start='2023-01-01', periods=24, freq='MS')

historico = pd.DataFrame({
    'mes': meses,
    'socios': np.random.poisson(lam=len(segmentos_df), size=24),
    'precio_promedio': np.random.normal(50, 5, size=24).round(2),
    'porc_puntuales': np.random.uniform(0.5, 0.7, size=24).round(2),
    'porc_retraso_leve': np.random.uniform(0.2, 0.3, size=24).round(2),
    'porc_morosos': np.random.uniform(0.05, 0.2, size=24).round(2),
})

# Simular ingresos con ponderación
historico['ingresos'] = (historico['socios'] * historico['precio_promedio'] *
                         (historico['porc_puntuales']*1 +
                          historico['porc_retraso_leve']*0.9 +
                          historico['porc_morosos']*0.7)).round(2)

historico.head()


In [ ]:

# Preparar datos para entrenamiento
X = historico[['socios', 'precio_promedio', 'porc_puntuales', 'porc_retraso_leve', 'porc_morosos']]
y = historico['ingresos']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

modelo = RandomForestRegressor(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

y_pred = modelo.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae:.2f}')
print(f'R2 Score: {r2:.2f}')


In [ ]:

# Predicción para el próximo mes hipotético
nuevo_mes = pd.DataFrame([{
    'socios': len(segmentos_df),
    'precio_promedio': 52,
    'porc_puntuales': 0.65,
    'porc_retraso_leve': 0.25,
    'porc_morosos': 0.1
}])

pred_ingreso = modelo.predict(nuevo_mes)
print(f'Ingreso estimado para el próximo mes: ${pred_ingreso[0]:.2f}')



## Conclusión

Con el modelo predictivo entrenado, es posible estimar el ingreso mensual futuro basado en:
- Cantidad de socios.
- Precio promedio.
- Distribución real de comportamiento de pago.

Este modelo puede integrarse en dashboards o herramientas de planificación financiera.
